<a href="https://colab.research.google.com/github/Nqam/store_report/blob/main/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем библиотеки для работы с таблицами

In [1]:
import pandas as pd
import numpy as np
import re

In [4]:
#Указываем путь на отчеты для каждой переменной - розничный и фабричный
#(можно загрузить файлы во временную библиотеку блокнота и провести все операции с ними)
rozn = '/content/drive/MyDrive/Розничный отчет Май 1-31.xlsx'
fabr = '/content/drive/MyDrive/Май 1-31 фабричный.xlsx'

Загружаем отчеты в память программы

In [5]:
%%time
rozn_t = pd.read_excel(rozn, skiprows=2) #это розничный отчет, пропускаются две строки, там шапка с объединенными ячейками
fabr_t = pd.read_excel(fabr) #это фабричный отчет

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CPU times: user 21 s, sys: 221 ms, total: 21.2 s
Wall time: 22.6 s


Переименовываем все столбцы к общему виду

In [6]:
rozn_t = rozn_t.rename(columns={'Модель': 'Дизайн',
                        'Цвет':'Цвет ткани',
                        'Название акции':'Акции',
                        'Структура на момент продажи':'Коллекция'})

fabr_t = fabr_t.rename(columns={'№':'Договор',
                        'от даты':'Дата',
                        'Основа':'Ткань',
                        'Оформил':'Продавец',
                        'Канал сбыта':'Канал продаж',
                        'Цвет основы':'Цвет ткани',
                        'Модель':'Дизайн',
                        'Название акции':'Акции',
                        'Заказчик':'Розничная точка'
                        })

#Добавляем недостающие для слияния столбцы
rozn_t['Источник_строк'] = 'Розничный отчет'
rozn_t['Канал продаж'] = ''
rozn_t['Скид. пост.'] = np.NaN
rozn_t['Скид. фабр.'] = np.NaN
rozn_t['Кат.'] = ''

fabr_t['Источник_строк'] = 'Фабричный отчет'
fabr_t['Заказчик'] = ''
fabr_t['Механизм трансформации'] = ''
fabr_t['Банковские продукты'] = ''
fabr_t['Название салона'] = ''
fabr_t['Юр. лицо'] = ''
fabr_t['Розничная точка'] = ''

In [7]:
rozn_t = rozn_t[[
  'Источник_строк','Акции','Банковские продукты','Бренд','Дата','Дизайн','Договор','Заказчик',
  'Канал продаж','Кат.','Категория заказчика','Комплектация','Коллекция','Механизм трансформации',
  'Название салона','Особые отметки','Пл.отгруз','Примечание','Продавец','Регион','Розничная точка',
  'Руководитель группы','Скид. пост.','Скид. прод.','Скид. фабр.','Сумма','Ткань','Цвет ткани',
  'Цена','Шт.','Юр. лицо']]

In [8]:
fabr_t = fabr_t[[
  'Источник_строк','Акции','Банковские продукты','Бренд','Дата','Дизайн','Договор',
  'Заказчик','Канал продаж','Кат.','Категория заказчика','Комплектация','Коллекция',
  'Механизм трансформации','Название салона','Особые отметки','Пл.отгруз','Примечание',
  'Продавец','Регион','Розничная точка','Руководитель группы','Скид. пост.','Скид. прод.',
  'Скид. фабр.','Сумма','Ткань','Цвет ткани','Цена','Шт.','Юр. лицо']]

In [9]:
df = pd.concat([rozn_t,fabr_t], ignore_index=True)
df = df[df['Дата'].isna() == False]
df['Шт. для MOON'] = ''

In [10]:
df['Диз+мех'] = ''
df['Спальное место'] = ''
df['Тип дивана'] = ''
df['Настил'] = ''
df['Категория заказчика'] = df['Категория заказчика'].astype(str)
df['Скид. прод.'] = df['Скид. прод.'].str.replace(',', '.').astype(float)

Удаляем модель 500 и точки содержащие "склад" и "интернет"

In [11]:
def int_skl(x):
  if pd.isna(x) == True:
    return x
  elif 'склад' in x.lower():
    return 'DROP_ROW'
  elif 'интернет' in x.lower() and 'ногинск' not in x.lower():
    return 'DROP_ROW'
  else:
    return x

df['Розничная точка_code'] = df['Розничная точка'].apply(int_skl)
df = df[(df['Розничная точка'] != 'DROP_ROW')&(df['Дизайн'] != '500')]
df = df.drop('Розничная точка_code', axis=1)

Меняем поле "дата" на нормальный тип

In [12]:
#from pandas.io.sql import date
def str_to_date(x):
  x = '.'.join([x[0:2],x[3:5],x[6:10]])
  return x

df['Дата'] = pd.to_datetime(df['Дата'].apply(str_to_date), dayfirst=True)

Убираем лишнее в поле "механизм трансформации" (МТ, кавычки)

In [13]:
def mechanism(x):
  if pd.isna(x) == True or x == '':
    return ''
  elif ('МТ ' in x or 'МТ ' in x) and x[-2] != ' ':
    return x[3:].replace('"','')
  elif re.search(r'^\bМТ',x) and x[-2] == ' ':
    return x[3:-2].replace('"','')
  elif not re.search(r'^\bМТ',x) and x[-2] == ' ':
    return x[:-2].replace('"','')
  else:
    return x

df['Механизм трансформации'] = df['Механизм трансформации'].apply(mechanism)

Убираем из примечаний и особых отметок "отказы" и "ошибочные"

In [14]:
df['Примечание'] = df['Примечание'].fillna('-')
df['Особые отметки'] = df['Особые отметки'].fillna('-')
df = df[(df['Примечание'].str.lower().str.contains('отказ', regex=True) == False)&(df['Примечание'].str.lower().str.contains('ошибочн', regex=True) == False)\
        &(df['Особые отметки'].str.lower().str.contains('отказ', regex=True) == False)|(df['Особые отметки'].str.lower().str.contains('ошибочн', regex=True) == False)]

Меняем бренды

In [15]:
#Меняю бренды на MOON или MOONFAMILY
def brand(x):
  if pd.isna(x) == True:
    return ''
  elif  'marketpl' in x.lower() and 'moonfamily' not in x.lower():
    return 'MOON'
  elif 'moonfamily' in x.lower() and 'marketpl' in x.lower():
    return 'MOONFAMILY'
  elif  x.lower() == 'marketplace':
    return 'MOONFAMILY'
  else:
    return x

df['Бренд'] = df['Бренд'].apply(brand)
df = df[df['Бренд'] != 'HOFF'] #Удаляю строки с брендом ХОФФ

In [16]:
df['Бренд'].unique()

array(['MOON', 'MOONFAMILY', ''], dtype=object)

Удаляем пустую или нулевую сумму заказа

In [17]:
df = df[(df['Сумма'] != 0) & (pd.isna(df['Сумма']) == False)]

Переименовать розничную точку ДИСКОНТ MOON_TRADE в Выставочный зал фабрики MF

Руководитель направления(группы) Осипова + Розничная точка Выставочный зал Ногинск - Выставочный зал фабрики MOON.(ДОДЕЛАТЬ)

Если Категория заказчика Оптовый – канал продаж Оптовый.
Категория заказчика Франчайзи М – Франчайзи М.
Категория заказчика Франчайзи МТ – Франчайзи MF.


In [18]:
def rozn(x):
  if pd.isna(x['Розничная точка']) == True:
    return ''
  elif 'оптовый' in x['Категория заказчика'].lower():
    return 'Оптовый'
  elif 'франчайзи м' in x['Категория заказчика'].lower():
    return 'Франчайзи М'
  elif 'франчайзи мт' in x['Категория заказчика'].lower():
    return 'Франчайзи MF'
  elif 'дисконт moon-trade' in x['Розничная точка'].lower():
    return 'Выставочный зал фабрики MF'
  elif 'кроват' in x['Розничная точка'].lower():
    return 'КиМ'
  elif 'осипова' in x['Руководитель группы'].lower() and 'Выставочный зал ногинск(интернет)' in x['Розничная точка']:
    return 'Выставочный зал фабрики MOON'
  elif 'михайлова' in x['Руководитель группы'].lower() and 'дисконт moon-trade' not in x['Розничная точка'].lower():
    return 'Розница MF'
  elif 'носкова' in x['Руководитель группы'].lower() and 'дисконт moon-trade' not in x['Розничная точка'].lower():
    return 'Розница M'
  else:
    return x['Розничная точка']

df['Канал продаж'] = df.apply(rozn, axis = 1)

In [19]:
df['Канал продаж'].unique()

array(['Выставочный зал Ногинск', 'Франчайзи М', 'Розница M', 'КиМ',
       'Розница MF', 'Оптовый', 'Выставочный зал Ногинск(интернет)',
       'Выставочный зал фабрики MF', ''], dtype=object)

1. Комплектация "41 " - Механизм трансформации "Бельевой короб"
2. Комплектация '41.','42.',"Т3","Т10" - Пантограф
3. Комплектация 50. и 60. - Форвард
4. Комплектация "Д2" - Дельфин
5. Комплектация Н10, Н20, Н30, Н12 - Аккордеон
6. Комплектация А10, А2, А3 - Еврокнижка
7. Комплектация содержит 12 - Кресла без механизма

In [20]:
#Задаю списки названий которые будут использоваться для замены
pant = ['41.','42.','Т3','Т10']
forw = ['50.','60. ']
accord = ['Н10', 'Н20', 'Н30', 'Н12']
eurobook = ['А10', 'А2', 'А3']


df['Комплектация'] = df['Комплектация'].str.replace('H10','Н10').str.replace('H20','Н20').str.replace('H30','Н30')\
  .str.replace('H12','Н12')
df['Комплектация'] = df['Комплектация'].str.replace('A10','А10').str.replace('A2','А2').str.replace('A3','А3')
df['Комплектация'] = df['Комплектация'].str.replace('T3','Т3').str.replace('T10','Т10')

#Проводим замену, в случае если элемент из списка встречается в комплектации - присваиваем мех. трансформации
def comp41(x):
  if pd.isna(x['Комплектация']) == True:
    return x['Механизм трансформации']
  elif '41 ' in x['Комплектация'] and pd.isna(x['Механизм трансформации']) == True:
    return 'Бельевой короб'
  elif any([i in x['Комплектация'] for i in pant]) and pd.isna(x['Механизм трансформации']) == True:
    return 'Пантограф'
  elif any([i in x['Комплектация'] for i in forw]) and pd.isna(x['Механизм трансформации']) == True:
    return 'Форвард'
  elif 'Д2' in x['Комплектация'] and pd.isna(x['Механизм трансформации']) == True:
    return 'Дельфин'
  elif any([i in x['Комплектация'] for i in accord]) and pd.isna(x['Механизм трансформации']) == True \
                 and '41Н10' not in x['Комплектация'] and '41Н20' not in x['Комплектация'] and '41Н30' not in x['Комплектация']:
    return 'Аккордеон'
  elif any([i in x['Комплектация'] for i in eurobook]) and pd.isna(x['Механизм трансформации']) == True:
    return 'Еврокнижка'
  elif '12' in x['Комплектация'] and pd.isna(x['Механизм трансформации']) == True:
    return 'Кресла без механизма'
  else:
    return x['Механизм трансформации']

df['Механизм трансформации'] = df.apply(comp41, axis = 1)

Добавляем новые столбцы (Диз+мех, Спальное место, Тип дивана, Настил)(ДОДЕЛАТЬ)

In [21]:
df['Диз+мех'] = df['Дизайн'] + ' ' + df['Механизм трансформации']

#Спальное место

Если Комплектация А10 х2 то Спальное место "160x200"

1. А21,A22,H10,H12 -  80x200
2. А27,A30,A32,H32,H32C -  160x200
3. Н20,H22 -  120x200
4. Н30 -  140x200
5. H32XL - 180x200




In [22]:
#Задаю списки названий которые будут использоваться для замены
eight = ['А21','А22','Н10','Н12']
six = ['А27','А30','А32','Н32C','Н32']
twelve = ['Н20','Н22']

#Задаем размеры спального места в зависимости от комплектации
def place(x):
  if pd.isna(x['Комплектация']) == True:
    return x['Спальное место']
  elif x['Комплектация'].count('А10') > 1:
    return '160x200'
  elif  any([i in x['Комплектация'] for i in eight]) :
    return '80x200'
  elif  any([i in x['Комплектация'] for i in twelve]):
    return '120x200'
  elif  'Н30' in x['Комплектация']:
    return '140x200'
  elif  any([i in x['Комплектация'] for i in six]):
    return '160x200'
  elif  'Н32XL' in x['Комплектация']:
    return '180x200'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Модель
1. 044, 123, 134 – 140x200
2. 110, 018, 109, 125, 127, 015 – 150x208
3. 107 – 150x210
4. 117 – 148x208

In [23]:
four = ['044','123','134'] #размер 140х200
five = ['110','018','109','125','127','015'] #размер 150х208

def place(x):
  if pd.isna(x['Дизайн']) == True:
    return x['Спальное место']
  elif  any([i in x['Дизайн'] for i in four]) and pd.isna(x['Спальное место']) == True:
    return '140x200'
  elif  any([i in x['Дизайн'] for i in five]) and pd.isna(x['Спальное место']) == True:
    return '150x208'
  elif  '107' in x['Дизайн'] and pd.isna(x['Спальное место']) == True:
    return '150x210'
  elif  '117' in x['Дизайн'] and pd.isna(x['Спальное место']) == True:
    return '148x208'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Пантографы

1.   135,139,140,141,142,143,144,145,146,173,174,183,184,185,180 – 140x200
2.   165, 171, 172, 175, 179 - 150x200
3.   007, 008,808, 015, 107, 111, 112, 160, 018, 109, 124, 125, 161, 162, 166 – 160x200

Пантограф

1.  Модель 160 и Т30 - 160x205
2.  Модель 176,178 если Т30 и 140 - 140x200
3.  если 150 - 150x200
4.  если 005, если А10 x2 - 160x200
5. если Т30 и 150 - 150x200 (перекрывается 3 пунктом)

Модель
1. Модель 032 - 80x190

In [24]:
four = ['135','139','140','141','142','143','144','145','146','173','174','183','184','185','180'] #размер 140х200
five = ['165','171','172','175','179'] #размер 150х200
six = ['007','008','808','015','107','111','112','160','018','109','124','125','161','162','166'] #размер 160х200

def place(x):
  if pd.isna(x['Дизайн']) == True:
    return x['Спальное место']
  elif  any([i in x['Дизайн'] for i in four]) and pd.isna(x['Спальное место']) == True and x['Механизм трансформации'] == 'Пантограф':
    return '140x200'
  elif  any([i in x['Дизайн'] for i in five]) and pd.isna(x['Спальное место']) == True and x['Механизм трансформации'] == 'Пантограф':
    return '150x200'
  elif  any([i in x['Дизайн'] for i in six]) and pd.isna(x['Спальное место']) == True and x['Механизм трансформации'] == 'Пантограф':
    return '160x200'

  elif  '160' in x['Дизайн'] and 'Т30' in x['Комплектация'] and pd.isna(x['Спальное место']) == True and x['Механизм трансформации'] == 'Пантограф':
    return '160x205'
  elif  ('176' in x['Дизайн'] or '178' in x['Дизайн'] or '140' in x['Дизайн']) and 'Т30' in x['Комплектация']\
   and pd.isna(x['Спальное место']) == True and x['Механизм трансформации'] == 'Пантограф':
    return '140x200'
  elif  '150' in x['Дизайн'] and pd.isna(x['Спальное место']) == True \
   and x['Механизм трансформации'] == 'Пантограф':
    return '150x200'
  elif  ('005' in x['Дизайн'] or x['Комплектация'].count('А10') > 1 or 'Т10' in x['Комплектация'])  and  pd.isna(x['Спальное место']) == True \
   and x['Механизм трансформации'] == 'Пантограф':
    return '160x200'
  elif  '032' in x['Дизайн'] and pd.isna(x['Спальное место']) == True:
   return '80x190'
  else:
    return x['Спальное место']


df['Спальное место'] = df.apply(place, axis = 1)

Аккордеоны
Все комплектации, где есть модули 7Я или 91,90,17 – угловой
Остальные – прямой


In [25]:
def corner(x):
  if pd.isna(x['Механизм трансформации']) == True:
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Аккордеон' and \
   (bool(re.search(r'(?<!\d)90(?!\d)',x['Комплектация'])) or bool(re.search(r'(?<!\d)91(?!\d)',x['Комплектация'])) or bool(re.search(r'(?<!\d)17(?!\d)',x['Комплектация'])) or bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower()))):
    return 'угловой'
  elif x['Механизм трансформации'] == 'Аккордеон' and x['Тип дивана'] == '':
    return 'прямой'

df['Тип дивана'] = df.apply(corner, axis=1)

Дельфины
Комплектации с Д - угловой (ВОПРОС)


In [26]:
df['Тип дивана'] = df['Механизм трансформации'].apply(lambda x: 'угловой' if x == 'Дельфин' else '') #что значит комплектация с Д

Еврокнижка
Если модуль 7Я - угловой,
А27 - угловой
Модели 039, 074 – прямые

In [27]:
def eurobook(x):
  if pd.isna(x['Механизм трансформации']) == True:
    return x['Тип дивана']
  elif x['Механизм трансформации'] == 'Еврокнижка' and (bool(re.search(r'(?<!\d)7я(?!\d)',x['Комплектация'].lower())) or bool(re.search(r'(?<!\d)А27(?!\d)',x['Комплектация'].lower()))):
    return 'угловой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Дизайн'] == '039':
    return 'угловой'
  elif x['Механизм трансформации'] == 'Еврокнижка' and x['Дизайн'] == '074':
    return 'прямой'
  else:
    return x['Тип дивана']

df['Тип дивана'] = df.apply(eurobook, axis=1)

005,007, 008, 160: комплектации с модулями Т7 7Я,7ПЯ,7ЛЯ, 76П(Л)Яа, 75П(Л)Яа (не путать с 17Я) и 90 – угловые, остальное – прямые (ДОДЕЛАТЬ)

007, 008, 160, 161, 162, 166 с модулями Т7 и 90,91 – угловые, остальное – прямые

Проставить настил.
Все комплектации с Lux – на НПБ, остальное – ППУ.


In [ ]:
df['Настил'] = df['Комплектация'].apply(lambda x: 'НПБ' if 'lux' in x.lower() else 'ППУ')

In [ ]:
#должны остаться только эти столбцы
df = df[['Договор','Дата','Заказчик','Категория заказчика','Канал продаж','Розничная точка','Дизайн','Диз+мех','Механизм трансформации',
'Комплектация','Спальное место','Тип дивана','Настил','Ткань','Цвет ткани','Кат.','Бренд','Сумма','Цена','Шт.','Шт. для MOON',
'Руководитель группы','Регион','Юр. лицо','Название салона','Скид. прод.','Акции','Продавец','Пл.отгруз','Примечание','Особые отметки',
'Банковские продукты','Коллекция','Скид. пост.','Скид. фабр.',]]

In [ ]:
#Сцепить Канал продаж, номер договора, розничная точка - через пробел, название столбца "Шт. для MOON"
df['Шт. для MOON'] = df['Канал продаж'] + ' ' + df['Договор'] + ' ' + df['Розничная точка']

In [ ]:
df.head()

,Договор,Дата,Заказчик,Категория заказчика,Канал продаж,Розничная точка,Дизайн,Диз+мех,Механизм трансформации,Комплектация,...,Скид. прод.,Акции,Продавец,Пл.отгруз,Примечание,Особые отметки,Банковские продукты,Коллекция,Скид. пост.,Скид. фабр.
0,ДПФ-5900,2023-05-02,Жувлакова Елена Владимировна,Розничный,Выставочный зал Ногинск,Выставочный зал Ногинск,071,071,,40х60 исп.1,...,0,NaN,Соголова Мария,02.05.2023,№компл.3131,-,КРЕДИТ ЕВРОПА БАНК ЗАО - Безналичный расчет,Продажа с подиума,NaN,NaN
1,ДПФ-5900,2023-05-02,Жувлакова Елена Владимировна,Розничный,Выставочный зал Ногинск,Выставочный зал Ногинск,071,071,,40х60 исп.1,...,0,NaN,Соголова Мария,02.05.2023,№компл.3133,-,КРЕДИТ ЕВРОПА БАНК ЗАО - Безналичный расчет,Продажа с подиума,NaN,NaN
6,ДПФ-63,2023-05-22,Березина Наталья Геннадьевна,Франчайзи МТ,Франчайзи М,"Ретюнская К.А., ИП, ТЦ ""Мебель парк"", г.Чита",№018,№018,,(декор подл+ков ВТ)Н30н+(2)65н+7Ян,...,0,NaN,NaN,22.05.2023,-,Без акции,NaN,Продажа с подиума,NaN,NaN
7,ДПФ-596,2023-05-27,Ивашечкин Олег,МООН,Розница M,"""МООН-М"", МЦ ""Большая Медведица"" (MOON) г. Кол...",123,123,,ник)(2)65а v1+(прав.подл,...,0,NaN,Ионина Галина Сергеевна,27.05.2023,-,Без акции,СБЕРБАНК РОССИИ ОАО - Безналичный расчет,Продажа с подиума,NaN,NaN
9,ДДФ-10677,2023-05-07,Сумникова Марина Валерьевна,Розничный,Выставочный зал Ногинск,Выставочный зал Ногинск,120,120,,12 H,...,0,NaN,Соголова Мария,13.05.2023,-,-,КРЕДИТ ЕВРОПА БАНК ЗАО - Безналичный расчет,NaN,NaN,NaN


In [ ]:
df.to_excel('res.xlsx')